In [11]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import copy

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, PCA
from sklearn.manifold import TSNE, MDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

Functions to output and save table, graph, and topic breakdowns given LDA model.

In [12]:
def topicBreakdown(df, lda):
    vocab = df.columns
    taxonomy = pd.read_csv('../hurwitzlab/data_sets/HMP_V13_taxonomy_fix.csv')
    result = ''

    for i, comp in enumerate(lda.components_):
        vocab_comp = zip(vocab, comp)
        sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
        result += ("Topic "+str(i)+": ")
        for t in sorted_words:
            row = taxonomy[taxonomy['OTU_ID'] == t[0]]
            if not row.empty:
                result += (str(row.iat[0, 6]) + ' ')
            else:
                result += ("Not Found")
        result += ("\n")

    print(result)

Original graph

In [13]:
body_sites = pd.read_csv('../hurwitzlab/data_sets/HMP_V13_participant_data.csv')

body_site_mapping = {site: idx for idx, site in enumerate(body_sites['HMP_BODY_SITE'].unique())}

body_site_ints = body_sites['HMP_BODY_SITE'].map(body_site_mapping)

In [14]:
df = pd.read_csv("../hurwitzlab/data_sets/HMP_V13_OTU_counts.csv")
df = df.drop(columns = ['PSN'])

# Standardize the data
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df)

# Initial dimensionality reduction
pca = PCA(n_components=50)
reduced_df = pca.fit_transform(scaled_df)

# Dimensionality reduction for visualization
tsne = TSNE(n_components=2, init='pca', random_state=0)
result = tsne.fit_transform(reduced_df)

In [ ]:
custom_colors = ['red', 'blue', 'green', 'yellow', 'purple']#, 'orange', 'pink', 'brown', 'olive', 'cyan']
cmap = ListedColormap(custom_colors)

# Plot with body sites
# Red = gut, blue = oral, green = airways, yellow = skin, purple = urogenital
fig = plt.figure(1, figsize=(8, 8))
plt.clf()
scatter = plt.scatter(result[:, 0], result[:, 1], c=body_site_ints, cmap=cmap, s=15)
plt.savefig("body_site_plot.svg")
plt.show()

In [29]:
def outputTableandGraph(df, lda, tax_level, filter):
    
    frequency_table = df.values
    topic_distributions = lda.transform(frequency_table)

    strongest_topic_indices = topic_distributions.argmax(axis=1)

    body_sites['Strongest_Topic'] = strongest_topic_indices

    topic_counts_by_site = body_sites.groupby(['HMP_BODY_SITE', 'Strongest_Topic']).size().unstack(fill_value=0)

    print(topic_counts_by_site)

    LDA_mapping = {site: idx for idx, site in enumerate(body_sites['Strongest_Topic'].unique())}

    LDA_ints = body_sites['Strongest_Topic'].map(LDA_mapping)

    custom_colors = ['red', 'blue', 'green', 'yellow', 'purple', 'orange', 'pink', 'brown', 'olive', 'cyan']
    cmap = ListedColormap(custom_colors)

    fig = plt.figure(1, figsize=(8, 8))
    plt.clf()
    scatter = plt.scatter(result[:, 0], result[:, 1], c=LDA_ints, cmap=cmap, s=15)
    plt.savefig(tax_level + "_comp_plot_" + str(filter) + ".svg")
    plt.show()

Function to find optimal component number for each taxonomic level by perplexity, returning a graph and a table at that level.

In [34]:
def findComponentNum(tax_level, file_name, filter):
    df = pd.read_csv('../hurwitzlab/data_sets/' + file_name + '.csv')
    df = df.drop(columns = ['PSN'])

    threshold = 10
    #df[df < threshold] = 0

    if (filter == 1):
      df = df.loc[:, (df != 0).sum(axis=0) > threshold]
    

    #remove taxa that occur less than a certain number of times across all samples (prevalence)
    #try things on mouth
    #try scikit clustering methods (hierarchical and k-means clustering)

    frequency_table = df.values

    bestLDA = LatentDirichletAllocation(n_components=5, random_state=0)
    bestLDA.fit(frequency_table)
    lowestPerplexity = bestLDA.perplexity(frequency_table)
    componentNum = 5
    print(componentNum, ',', lowestPerplexity, '\n')
    componentNum += 1


    decreasing = 1

    while decreasing == 1 and componentNum <= 10:
      LDA = LatentDirichletAllocation(n_components=componentNum, random_state=0)
      LDA.fit(frequency_table)
      perplexity = LDA.perplexity(frequency_table)

      print(componentNum, ', ', perplexity, '\n')
      
      if perplexity < lowestPerplexity:
        bestLDA = copy.deepcopy(LDA)
        lowestPerplexity = perplexity
      else:
        decreasing = 0

      componentNum += 1

    print(tax_level, '- Component number:', componentNum - 2 + decreasing, ', Perplexity:', lowestPerplexity, '\n')

    return bestLDA, df

Combined function

In [35]:
def completeAnalysis(tax_level, file_name, filter):
    lda, df = findComponentNum(tax_level, file_name, filter)
    #topicBreakdown(df, lda)
    outputTableandGraph(df, lda, file_name, filter)
    return df, lda

In [ ]:
Phylum_df, Phylum_lda = completeAnalysis('Phylum', 'HMP_V13_Phylum_counts', 0)

In [ ]:
Phylum_df, Phylum_lda = completeAnalysis('Phylum', 'HMP_V13_Phylum_counts', 1)

In [ ]:
OTU_df, OTU_lda = completeAnalysis('OTU', 'HMP_V13_OTU_counts', 0)

In [ ]:
OTU_df, OTU_lda = completeAnalysis('OTU', 'HMP_V13_OTU_counts', 1)

In [40]:
#genus_df, genus_lda = completeAnalysis('genus', 'HMP_V13_genus_counts')

In [ ]:
family_df, family_lda = completeAnalysis('family', 'HMP_V13_family_counts', 0)

In [ ]:
family_df, family_lda = completeAnalysis('family', 'HMP_V13_family_counts', 1)

In [43]:
#order_df, order_lda = completeAnalysis('order', 'HMP_V13_order_counts')

In [44]:
#class_df, class_lda = completeAnalysis('class', 'HMP_V13_class_counts')